# Load libraries

In [ ]:
# import [package name] as [name abbr.]
# python 處理數字跟 tensor 運算的主要套件
import numpy as np
# google 的 NN coding 套件
import tensorflow as tf

# 有可能會出現警告，但是可以不用理他
# c:\users\silver\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36:
# FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating`
# is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
# from ._conv import register_converters as _register_converters

import matplotlib.pyplot as plt

# Load Dataset

In [ ]:
exec(open("00_readingInput.py").read())
feature_dim = X_train.shape[1]
classes = 5

In [ ]:
# Define Model Input (x) and Output (y_),  y_ = f(x)
x = tf.placeholder(tf.float32, [None, X_train.shape[1]])
y_ = tf.placeholder(tf.int32, [None])
y_one = tf.one_hot(y_,classes)

In [ ]:
adam_h1 = tf.layers.dense(x,128,activation=None)
adam_h1_act = tf.nn.relu(adam_h1)
adam_h2 = tf.layers.dense(adam_h1_act,256,activation=None)
adam_h2_act = tf.nn.relu(adam_h2)
adam_y = tf.layers.dense(adam_h2_act,classes,activation=None)

# Define the Model Loss (4)
adam_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_one, logits=adam_y))
adam_train_step = tf.train.AdamOptimizer(0.001).minimize(adam_loss)

# Accuracy of the Model
adam_y_pred = tf.argmax(tf.nn.softmax(adam_y), 1, output_type=tf.int32)
adam_correct = tf.equal(adam_y_pred, y_)
adam_accuracy = tf.reduce_mean(tf.cast(adam_correct, tf.float32))

In [ ]:
# initialize the models
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Training & Evaluation

In [ ]:
epoch = 50
batch_size = 16
get_batches = gen_batches(X_train, Y_train, batch_size)
batches_in_a_epoch = int(X_train.shape[0]/batch_size)

In [ ]:
adam_hist_train_acc = []
adam_hist_valid_acc = []
adam_hist_train_loss = []
adam_hist_valid_loss = []

for step in range(epoch*batches_in_a_epoch):
    batch_xs, batch_ys = next(get_batches)
    sess.run(adam_train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if (step % batches_in_a_epoch == 0):
        # trainin and validation evaluation
        train_loss, train_acc = sess.run([adam_loss,adam_accuracy], feed_dict={x: X_train, y_: Y_train})
        valid_loss, valid_acc = sess.run([adam_loss,adam_accuracy], feed_dict={x: X_test,  y_: Y_test})
        adam_hist_train_acc.append(train_acc)
        adam_hist_valid_acc.append(valid_acc)
        adam_hist_train_loss.append(train_loss)
        adam_hist_valid_loss.append(valid_loss)
        #print("Accuracy: [T] %.4f / [V] %.4f" % (train_acc,valid_acc))
sess.close()

In [ ]:
plt.figure(0,figsize=(8,6))
plt.subplot(121)
x = [x for x in range(len(adam_hist_valid_loss))]
line_train, = plt.plot(x, adam_hist_train_loss, label='Training')
line_valid, = plt.plot(x, adam_hist_valid_loss, label='Validation')
plt.xlabel('#epoch')
plt.ylabel('Loss')
plt.legend(handles=[line_train,line_valid], loc=4)
plt.subplot(122)
x = [x for x in range(len(adam_hist_valid_acc))]
line_train, = plt.plot(x, adam_hist_train_acc, label='Training')
line_valid, = plt.plot(x, adam_hist_valid_acc, label='Validation')
plt.xlabel('#epoch')
plt.ylabel('Accuracy')
plt.legend(handles=[line_train,line_valid], loc=4)
plt.tight_layout()
plt.savefig('./training_curve/ex05_OverfittingCheck.png',dpi=300,format='png')
plt.show()
plt.close()